In [1]:
using JuMP, GLPK
using DataFrames
using CSV

In [2]:
##Creation of the Input DataFrames

#load demand data
Demands = CSV.File("../Max/Data/Demands.csv") |> DataFrame

#load conventional generators data
Conv_gen = CSV.File("../Max/Data/Conventional_generators.csv") |> DataFrame

#load wind farms data
Wind_farms = CSV.File("../Max/Data/Wind_farms.csv") |> DataFrame

# load wind profile
wind_profile = CSV.File("../data/wind_powerprofile_200.csv") |> DataFrame


Row,Hour,W1,W2,W3,W4,W5,W6
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,76.8921,101.54,92.8003,95.3709,96.002,70.9073
2,2,66.8277,90.9989,109.167,107.937,103.616,112.618
3,3,78.422,116.959,142.88,134.78,128.276,133.832
4,4,64.1437,134.249,159.57,133.664,142.969,154.233
5,5,102.22,145.507,160.944,165.303,144.03,161.49
6,6,134.039,131.112,157.023,161.828,144.636,155.222
7,7,146.517,153.67,141.566,159.927,152.86,153.604
8,8,143.176,163.392,155.294,170.805,158.643,164.307
9,9,163.297,144.48,172.638,185.318,150.901,166.558


In [6]:
# number of convential generators
G = size(Conv_gen, 1)

# number of demand
D = size(Demands, 1)

# number of wind generators
W = size(Wind_farms, 1)

4

In [7]:
##Definition of the Result DataFrames

# Definition of the Main result DataFrame
main_result = DataFrame(hour = Int[], social_welfare = Float64[], market_price = Float64[])

# Initialize the Demand DataFrame directly without dynamic column names
details_results_demand = DataFrame(hour = Int[], total_demand = Float64[])
# For x variables, manually add each column. This is a one-time setup.
for i in 1:D
    details_results_demand[!, Symbol("demand_n°$i")] = Float64[]
end

# Initialize the Supply DataFrame directly without dynamic column names
details_results_supply = DataFrame(hour = Int[], total_supply = Float64[])
# For x variables, manually add each column. This is a one-time setup.
for i in 1:G
    details_results_supply[!, Symbol("supply_n°$i")] = Float64[]
end
for i in G:G+W
    details_results_supply[!, Symbol("supply_n°$i")] = Float64[]
end


In [8]:
##Definition of the model

# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [11]:
#Clearing the DataFrames
empty!(main_result)
empty!(details_results_demand)
empty!(details_results_supply)

hour = 1
# Define the decision variables for every supply and every demand
unregister(model, :x)  # Unregister the existing variable named "x" from the model
unregister(model, :BalanceConstraint)
@variable(model, x[1:G + W + D])  # Create the new variables

for i in 1:G
    set_lower_bound(x[i], 0)  # Set the lower bound
    set_upper_bound(x[i], Conv_gen[i, 6])  # Set the upper bound
end

for i in (G + 1):(G + W)
    set_lower_bound(x[i], 0)  # Set the lower bound
    set_upper_bound(x[i], wind_profile[hour + 1, i - G + 1])  # Set the upper bound
end

for i in (G + W + 1):(G + W + D)
    set_lower_bound(x[i], 0)  # Set the lower bound
    set_upper_bound(x[i], Demands[i - G - W, 3])  # Set the upper bound
end

# Define the objective function
@objective(model, Min,
    - (sum(Demands[demand, 4] * x[demand + G + W] for demand in 1:D)
    - (sum(Conv_gen[generator, 3] * x[generator] for generator in 1:G)
    + sum(0 * x[generator + G] for generator in 1:W))))

# Add the equilibrium constraint
@constraint(model, BalanceConstraint, sum(x[generator] for generator in 1:G + W) == sum(x[demand + G + W] for demand in 1:D))

# Solve the model
optimize!(model)

status = termination_status(model)
if status == MOI.OPTIMAL
    println("Hour ", hour, ": Optimal solution found")

    # Assuming 'model' and 'x' are properly defined and 'hour' is known
    new_mainrow = [hour, -objective_value(model), dual(BalanceConstraint)]
    push!(main_result, new_mainrow)

    new_demandrow = [hour, sum(value(x[i + G + W]) for i in 1:D)]
    append!(new_demandrow, [value(x[i + G + W]) for i in 1:D])
    push!(details_results_demand, new_demandrow)

    new_supplyrow = [hour, sum(value(x[i]) for i in 1:G + W)]
    append!(new_supplyrow, [value(x[i]) for i in 1:G + W])
    push!(details_results_supply, new_supplyrow)

elseif status == MOI.INFEASIBLE
    println("Hour ", hour, ": No feasible solution found")
else
    println("Hour ", hour, ": Solver status: ", status)
end


    

Hour 1: Optimal solution found


Row,hour,total_supply,supply_n°1,supply_n°2,supply_n°3,supply_n°4,supply_n°5,supply_n°6,supply_n°7,supply_n°8,supply_n°9,supply_n°10,supply_n°11,supply_n°12,supply_n°13,supply_n°14,supply_n°15,supply_n°16
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,2207.0,106.4,106.4,170.269,0.0,0.0,108.5,108.5,280.0,280.0,210.0,217.0,245.0,66.8277,90.9989,109.167,107.937


In [12]:
#save result_df to csv
CSV.write("../Max/Results/Task1/main_result_Max", main_result)
CSV.write("../Max/Results/Task1/details_results_demand_Max", details_results_demand)
CSV.write("../Max/Results/Task1/details_results_supply_Max", details_results_supply)


"../Max/Results/Task1/details_results_supply_Max"